
# Analyze "Russia Housing Prices" and predicting them

The goal with the given dataset is to use the data to make a prediction-model after doing an EDA. To achieve this I first come up with some questions I want to answer. Furthermore, I will be using a CRISP-DM like process to get from the EDA to the prediction-model.

## 1. Business Understanding

This phase is used to generate the questions I want to answer. These are the following:
1. Are there some places listed more than once in the period of time provided?
2. What is the average price per m²? What regions have the highest average?
3. How did the prices change over the years?

Other than answering these questions, the goal is also to predict the prices with a fitting ML-Algorithm.


## 2. Data Understanding
 For the data understanding the best thing is to use the description of the kaggle-side where the data is from (https://www.kaggle.com/datasets/mrdaniilak/russia-real-estate-20182021). This saves quite some time.

The dataset has 13 fields. Each row contains data about an ad for real estate in russia.

- **date**: date of publication of the announcement.
- **time**: the time when the ad was published.
- **geo_lat**: Latitude.
- **geo_lon**: Longitude.
- **region**: Region of Russia. There are 85 subjects in the country in total.
- **building_type**: Facade type. 0 - Other. 1 - Panel. 2 - Monolithic. 3 - Brick. 4 - Blocky. 5 - Wooden.
- **object_type**: Apartment type. 1 - Secondary real estate market; 2 - New building;.
- **level**: Apartment floor.
- **levels**: Number of storeys.
- **rooms**: the number of living rooms. If the value is "-1", then it means "studio apartment".
- **area**: the total area of the apartment in m².
- **kitchen_area**: - Kitchen area in m².
- **price**: Price in rubles.


In [45]:
import dateutil.utils
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from currency_converter import CurrencyConverter
c = CurrencyConverter()

import warnings
warnings.filterwarnings('ignore')



In [46]:
df = pd.read_csv('all_v2.csv')
df.date = pd.to_datetime(df.date)
df.date = df.date.dt.date
temp = pd.to_datetime(df.time, format='%H:%M:%S').apply(lambda d : d.time())
df.time = temp

In [47]:
df

,price,date,time,geo_lat,geo_lon,region,building_type,level,levels,rooms,area,kitchen_area,object_type
0,6050000,2018-02-19,20:00:21,59.805808,30.376141,2661,1,8,10,3,82.6,10.8,1
1,8650000,2018-02-27,12:04:54,55.683807,37.297405,81,3,5,24,2,69.1,12.0,1
2,4000000,2018-02-28,15:44:00,56.295250,44.061637,2871,1,5,9,3,66.0,10.0,1
3,1850000,2018-03-01,11:24:52,44.996132,39.074783,2843,4,12,16,2,38.0,5.0,11
4,5450000,2018-03-01,17:42:43,55.918767,37.984642,81,3,13,14,2,60.0,10.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477001,19739760,2021-05-01,20:13:58,55.804736,37.750898,3,1,8,17,4,93.2,13.8,11
5477002,12503160,2021-05-01,20:14:01,55.841415,37.489624,3,2,17,32,2,45.9,6.6,11
5477003,8800000,2021-05-01,20:14:04,56.283909,44.075408,2871,2,4,17,3,86.5,11.8,1
5477004,11831910,2021-05-01,20:14:12,55.804736,37.750898,3,1,8,33,2,52.1,18.9,11


In [48]:
df.describe(datetime_is_numeric=True)

,price,geo_lat,geo_lon,region,building_type,level,levels,rooms,area,kitchen_area,object_type
count,5.477006e+06,5.477006e+06,5.477006e+06,5.477006e+06,5.477006e+06,5.477006e+06,5.477006e+06,5.477006e+06,5.477006e+06,5.477006e+06,5.477006e+06
mean,4.422029e+06,5.403826e+01,5.324433e+01,4.307141e+03,1.948966e+00,6.214530e+00,1.139892e+01,1.726173e+00,5.391825e+01,1.062840e+01,3.945399e+00
std,2.150752e+07,4.622758e+00,2.074763e+01,3.308050e+03,1.038537e+00,4.957419e+00,6.535734e+00,1.082133e+00,3.335293e+01,9.792380e+00,4.558357e+00
min,-2.144967e+09,4.145906e+01,1.989020e+01,3.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,-2.000000e+00,7.000000e-02,1.000000e-02,1.000000e+00
25%,1.950000e+06,5.337768e+01,3.777790e+01,2.661000e+03,1.000000e+00,2.000000e+00,5.000000e+00,1.000000e+00,3.800000e+01,7.000000e+00,1.000000e+00
50%,2.990000e+06,5.517139e+01,4.306774e+01,2.922000e+03,2.000000e+00,5.000000e+00,1.000000e+01,2.000000e+00,4.802000e+01,9.700000e+00,1.000000e+00
75%,4.802000e+06,5.622613e+01,6.564895e+01,6.171000e+03,3.000000e+00,9.000000e+00,1.600000e+01,2.000000e+00,6.313000e+01,1.270000e+01,1.100000e+01
max,2.147484e+09,7.198040e+01,1.625361e+02,6.188800e+04,5.000000e+00,3.900000e+01,3.900000e+01,1.000000e+01,7.856000e+03,9.999000e+03,1.100000e+01


In [49]:
df.object_type.value_counts()

1     3863809
11    1613197
Name: object_type, dtype: int64

In [50]:
df.isnull().sum()

price            0
date             0
time             0
geo_lat          0
geo_lon          0
region           0
building_type    0
level            0
levels           0
rooms            0
area             0
kitchen_area     0
object_type      0
dtype: int64

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5477006 entries, 0 to 5477005
Data columns (total 13 columns):
 #   Column         Dtype  
---  ------         -----  
 0   price          int64  
 1   date           object 
 2   time           object 
 3   geo_lat        float64
 4   geo_lon        float64
 5   region         int64  
 6   building_type  int64  
 7   level          int64  
 8   levels         int64  
 9   rooms          int64  
 10  area           float64
 11  kitchen_area   float64
 12  object_type    int64  
dtypes: float64(4), int64(7), object(2)
memory usage: 543.2+ MB


With the code blocks above, we can measure the data quality:
- **Correctness**: The data is not entirely correct since we got e.g. prices that are negative and object_type seems not to be quite correct since they do not match with the given description. These can be fixed in the data preparation phase.
- **Completeness**: The data seems to be complete.
- **Redundancy**: For now I cannot spot any redundancies.

## 3. Data Preparation

After spotting some flaws in the data understanding phase, we tackle those in the upcoming phase. The first thing to be done is deleting the negative prices and replacing the 11 with 2 in the `object_type`.

In [52]:
df = df[df.price>0]

In [53]:
df.object_type = df.object_type.replace(11, 2)

In [54]:
df.object_type.unique()

array([1, 2], dtype=int64)

The other problem we have is that the price is represented in russian rubels, which is obvious since we got a dataset about russian housing prices. For me it still makes much more sense to analyze the data in euro. Because we have a date for each entry which ranges from spring 2018 to early summer 2021, we need different exchange rates from those years. I acquired them from the website of the European Central Bank (https://sdw.ecb.europa.eu/quickview.do?SERIES_KEY=120.EXR.D.RUB.EUR.SP00.A).

In [55]:
df_exchange = pd.read_csv('amCharts.csv', sep= ';')
df_exchange = df_exchange.rename(columns={'s1':'rate'})
df_exchange.date = pd.to_datetime(df_exchange.date)
df_exchange.date = df_exchange.date.dt.date

Next up we have to convert the rate to an actual rate, because now it just represents the value of 1 euro in rubel.

In [56]:
df_exchange.rate = 1/df_exchange.rate

Now we map the exchange rate to the housing price df. After that the price gets transformed to euro.

In [57]:
df = df.merge(df_exchange, left_on='date', right_on='date')
df.price = df.price * df.rate
df.price = df.price.round(2)
df.price

0           86341.12
1          125731.31
2           58178.43
3           26742.30
4           78781.36
             ...    
4325173     49608.21
4325174    115752.48
4325175     34174.54
4325176     48505.80
4325177    289932.40
Name: price, Length: 4325178, dtype: float64

19.9030946